## Init & utils

In [1]:
%%bash
pip install --upgrade pip
pip install jsonpath-rw
pip install jsonpath-rw-ext
pip install jsondiff
pip install seaborn

Requirement already up-to-date: pip in /srv/conda/lib/python3.6/site-packages (18.0)
  Running setup.py bdist_wheel for jsonpath-rw: started
  Running setup.py bdist_wheel for jsonpath-rw: finished with status 'done'
  Stored in directory: /home/jovyan/.cache/pip/wheels/5c/00/9a/82822db383c2d96dcebf839786665a185f92d37e5026f9806f
Successfully built jsonpath-rw
  Running setup.py bdist_wheel for jsondiff: started
  Running setup.py bdist_wheel for jsondiff: finished with status 'done'
  Stored in directory: /home/jovyan/.cache/pip/wheels/46/5f/86/11c6b72b064888e80b98bfcbcdaf2a83517a8cf8f2bb2a3227
Successfully built jsondiff


In [18]:
%ls -l

total 24700
-rw-r--r-- 1 jovyan jovyan        8 Sep 17 09:34 application.properties
-rw-r--r-- 1 jovyan jovyan   258347 Sep 28 22:29 bga.puertorico.analyze.ipynb
-rw-r--r-- 1 jovyan jovyan    21055 Sep 28 22:14 bga.puertorico.crawl.ipynb
drwxr-xr-x 1 jovyan jovyan     4096 Sep 17 03:43 conda-bld/
drwxr-xr-x 2 jovyan jovyan     4096 Sep 27 19:55 pr/
-rw-r--r-- 1 jovyan jovyan 18130930 Sep 28 22:29 pr_4.tgz
-rw-r--r-- 1 jovyan jovyan  6854990 Sep 28 22:17 pr_tables.tgz


In [21]:
%%bash
# tar -cvzf pr_4.tgz pr
# tar -cvzf pr_tables.tgz pr_tables.json
# tar -xvzf pr_tables.tgz
# tar -xvzf pr_4.tgz

In [1]:
from jsonpath_rw import jsonpath, parse
import jsonpath_rw_ext as jsonp
import collections
import os

In [2]:
import json 
from json import JSONEncoder, JSONDecoder
import pickle

class PythonObjectEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (list, dict, str, unicode, int, float, bool, type(None))):
            return JSONEncoder.default(self, obj)
        return {'_python_object': pickle.dumps(obj)}

def as_python_object(dct):
    if '_python_object' in dct:
        return pickle.loads(str(dct['_python_object']))
    return dct

def save_to_json_file(filename, content):
    temp = json.dumps(content, cls=PythonObjectEncoder)
    return save_to_file(filename, temp)

def save_to_file(filename, content):
    with open(filename, 'w') as file:
        file.write(content)
    return "saved " + filename    

def load_from_json_file(file_name):
    content = load_from_file(file_name)
    return json.load(content, object_hook=as_python_object)

def load_from_file(file_name):
    return open(file_name)

In [3]:
import copy

def inc_dict_val(d, k, v):
    d[k] = int(d.get(k, 0)) + int(v)

def alter_dict(D, **F):
    result = copy.deepcopy(D)    
    result.update(**F)
    return result   

def map_dict(src, *args, **kvargs):
    dst = dict()
    for v in args:
        dst[v] = src[v]
    for k,v in kvargs.items():
        dst[k] = src[v]
    return dst

def remove_from_dict(d, key):
    if key in d:
        del d[key]

## Printing

In [4]:
from pprint import pprint
from IPython.display import JSON

width = 180
def print_line(char=' '):
    print(char*width)

In [5]:
import re

def print_log(log, consumer=print, **args):
    if log:
        params = {g:'' for g in re.findall('\$\{([^}]+)\}', log)}
        params.update(args)
        consumer(log.replace('$', '').format(**params))

In [6]:
from jsondiff import diff

def print_diff(old_dict, new_dict, consumer=print):     
    consumer(get_diff(old_dict, new_dict))
    
def get_diff(old_dict, new_dict):     
    return diff(old_dict, new_dict, syntax='symmetric')

In [7]:
points = [1, 5, 20, 100, 500]

import sys

def print_progress(i):
    if (i % points[0] == 0):
        if (i % points[1] == 0):
            sys.stdout.write(' ')
            if (i % points[2] == 0):
                sys.stdout.write('  ')
                if (i % points[3] == 0):
                    sys.stdout.write('\n')
                    if (i % points[4] == 0):
                        sys.stdout.write('\n')
        sys.stdout.write('.')
        sys.stdout.flush()                        

## Analitical fn

In [9]:
def collect_data(files, collector):
    for i,f in enumerate(files):
        data = load_from_json_file(f)
        print_progress(i)
        yield collector(data)

In [10]:
def select_all(key):
    return lambda data: jsonp.match(key, data)

def select_one(key):
    return lambda data: jsonp.match1(key, data)

def contains(key, value):
    return lambda data: jsonp.match1(key, data) == value

In [18]:
def all_of(*filters):
    def fn(data):
        for f in filters:
            if not f(data):
                return False
        return True  
    return fn

In [12]:
def flattern(data):
    return [l2 for l1 in data for l2 in l1]

In [11]:
def get_nodes(data, ptr):
    return [match.value for match in jsonp.parse(ptr).find(data)]

## Analysis

In [8]:
from os import listdir
from os.path import isfile, join

game = 'pr'
games = [join(game, f) for f in listdir(game) if f.endswith('.json')]
len(games)

171

In [13]:
infos = flattern(collect_data(games, select_all('$.info')))
JSON(infos[:5])

   

..... ..... ..... .....   ..... ..... ..... .....   ..... ..... ..... .....   ..... ..... ..... .....   ..... ..... ..... .....   
..... ..... ..... .....   ..... ..... ..... .....   ..... ..... ..... .....   ..... ..... .

<IPython.core.display.JSON object>

In [14]:
opts = list(map(select_one('$.options'), infos))
JSON(opts[:5])

<IPython.core.display.JSON object>

In [15]:
get = lambda l, x: l[int(x)] if isinstance(l, list) else l[x]

In [19]:
# contains('$.info.result.endgame_reason', 'normal_end')
fi = list(filter(all_of(
    contains('$."100".value', '0'),
    contains('$."101".value', '0'),
    contains('$."102".value', '0'),
    contains('$."103".value', '0')                      
), opts))

for o in ['100', '101', '102', '103']:
    l = list(map(lambda v: get(v['values'], v['value'])['name'], map(lambda v: v[o], fi)))

    c = collections.Counter(l)
    print(o, opts[0][o]['name'], c.most_common(20))

100 Balance variant [('Classic game', 84)]
101 Two-player variant [('Classic game', 84)]
102 First expansion - New buildings [('Off', 84)]
103 Second expansion - The nobles [('Off', 84)]


In [20]:
JSON(list(filter(contains('$.options."101".value', '1'), infos)))

<IPython.core.display.JSON object>

## Generic bga functions

In [ ]:
def load_game(fn):
    return load_from_json_file(fn)['data']['data']['data']

In [ ]:
def get_step(game, move_id=0, sub_move_id=0, **kvargs):
    return game[move_id]['data'][sub_move_id]

In [ ]:
def move_to_next_step(game, game_state):
    move_id     = game_state.get('move_id', 0)
    sub_move_id = game_state.get('sub_move_id', 0) + 1
    
    if len(game[move_id]['data']) <= sub_move_id:
        move_id    += 1
        sub_move_id = 0
    
    return alter_dict(game_state,
        move_id     = move_id,
        sub_move_id = sub_move_id
    )

In [ ]:
def default_action(game_state, step):
    print("Unknown action '%s' :(" % step['type'])
    display(JSON({'state': game_state, 'step': step}, indent=4))
    return None

In [ ]:
def apply_step(game_state, step):
    if not step['args']: 
        step['args'] = dict()
    elif not isinstance(step['args'], dict): 
        step['args'] = {str(i):v for i,v in enumerate(step['args'])}
        
    if step['log']:
        print_log('[{step_type}] ' + step['log'], step_type=step['type'], **step['args'])
        
    a = actions.get(step['type'], None)
    if a:
        return a(game_state, **step['args'])
    else:
        return default_action(game_state, step)        

In [ ]:
def debug(state, step, fn=None):
    if not fn:
        fn = globals()[step['type']]
    display(
        JSON(state, root='state'), 
        JSON(step, root='step'), 
    )
    new_state = fn(state, **step['args'])
    pprint(get_diff(state, new_state), width=10)

## Puerto rico functions

## initialization

In [ ]:
good_types = ['corn', 'indigo', 'sugar', 'tobacco', 'coffee']

def setup_buildings(buildings, **kvargs):
    result = {}
    def inc(d, k): d[k] = 1 + d.get(k, 0) 
    for b in buildings.values():
        inc(result, b['bld_type'])
    return result

def setup_plantations(plantations_to_set, **kvargs):
    result = [p['res_type'] for p in plantations_to_set.values()]
    return result

def setup_ships(ships, **kvargs): 
    result = {v['shp_id']:{
        'capacity': int(v['shp_capacity']),
        'goods': 0,
        'res_type': None
    } for v in ships.values()}
    return result

def setup_stock(**kvargs):
    result = []
    return result

def setup_VPs(counters, **kvargs): 
    return int(counters['shipping_points_bank']['counter_value'])

def setup_bank(counters, **kvargs): 
    return int(counters['doubloons_bank']['counter_value'])

def setup_workers(counters, **kvargs): 
    result = {
        'available': int(counters['colonists_bank']['counter_value']),
        'arriving':  int(counters['colonists_ship']['counter_value'])
    }
    return result

def setup_goods(counters, **kvargs): 
    result = {
        'coffee'  : int(counters['resource_coffee_bank']['counter_value']),
        'corn'    : int(counters['resource_corn_bank']['counter_value']),
        'indigo'  : int(counters['resource_indigo_bank']['counter_value']),
        'sugar'   : int(counters['resource_sugar_bank']['counter_value']),
        'tobacco' : int(counters['resource_tobacco_bank']['counter_value']),
    }
    return result

def setup_roles(role_tooltips, **kvargs):
    result = {v['rol_id']:{
        'money': int(v['money']), 
        'rol_type': v['rol_type'], 
        'selected': False
    } for v in role_tooltips.values()}
    return result
    
def setup_playerorder(playerorder, **kvargs):
    result = [str(v) for v in playerorder]
    return result

def setup_player(index, username, money, plantaion, **kvargs):
    result = {
        '_': username,
        'money': int(money),
        'building_vp': 0,
        'delivery_vp': 0,
        'buildings': {'commercial': {}, 'factories': {g:{'jobs':0, 'workers':0} for g in good_types}},
        'goods': {g:0 for g in good_types},
        'plantations': {g:{'jobs':1 if g == plantaion else 0, 'workers':0}  for g in (good_types + ['rock'])},
        'potential_vp': {'commercial': 0, 'factories': 0, 'workers': 0},
        'slots': {'buildings': 12, 'plantations': 11},
        'workers': {'total': 0, 'unemployed': 0}
    }
    result['goods']['stored'] = {}
    return result

def setup_players(players, playerorder, counters, plantations_owned, **kvargs):
    result = {}
    for index,id in enumerate(playerorder):
        id = str(id)
        money = counters['doubloons_player_'+id]['counter_value']
        plantation = [v['res_type'] for k,v in plantations_owned.items() if v['player_id'] == id][0]
        result[id] = setup_player(index, players[id]['name'], money, plantation)
    return result

def setup_table(datas):
    r = {
        'buildings': setup_buildings(**datas),
        'plantations': setup_plantations(**datas),
        'ships': setup_ships(**datas),
        'stock': setup_stock(**datas),
        'VPs': setup_VPs(**datas),
        'money': setup_bank(**datas),
        'workers': setup_workers(**datas),
        'goods': setup_goods(**datas),
        'roles': setup_roles(**datas),
        'playerorder': setup_playerorder(**datas)
    }
    return r

def setup(datas):
    return {
        'move_id': 0,
        'sub_move_id': 0,
        'table': setup_table(datas),
        'players': setup_players(**datas),
        'governor': datas['governor']
    }

In [ ]:
def setup_lookups(buildings, plantations_owned, **kvargs):
    global building_lookup
    building_lookup = {k:{
        'name': b['bld_type'],
        'type': 'factories' if b['res_type'] else 'commercial',
        'res_type': b['res_type'],
        'potential_vp': 1 if b['bld_valence'] == "1" else 2,
        'tile_size': 2 if b['bld_max_discount'] == "4" else 1,
        'jobs': int(b['bld_valence']),
        'cost': int(b['bld_cost']),
        'max_discount': int(b['bld_max_discount']),
        'vp': int(b['bld_score'])
    } for k,b in buildings.items()}
    
    global plantation_lookup
    plantation_lookup = {k:p['res_type'] for k,p in plantations_owned.items()}
    
    global colonist_lookup
    colonist_lookup = {}    
    
    global run_limits
    run_limits = {}   

In [ ]:
new_game_state = 1
if not 'actions' in globals():
    global actions
    actions = dict()    

game_log = load_game('table_27174212.json')
datas = load_from_json_file('datas_27174212.json')[7]
game_state = setup(datas)
setup_lookups(**datas)

### Unverified

In [ ]:
def get_player(state, player_id=None, **args):
    return state['players'].get(str(player_id), None)

def prepare(state, **kvargs):
    new_state = alter_dict(state)
    player = get_player(new_state, **kvargs)
    table = new_state['table']
    return new_state, player, table

In [ ]:
def do_nothing(log=None):
    def dn(state, **args):
        print_log(log, **args)
        return state
    return dn
log = ' - d:${description:70} t:${type}, n:${name}'#', a:${action}'#', possible_actions:${possibleactions}'

actions['leaveGameState']      = do_nothing('<-[leaveGameState ]' + log)
actions['gameStateChange']     = do_nothing('->[gameStateChange]' + log)
actions['updateReflexionTime'] = do_nothing('  [updateReflexionTime]')
actions['message' ]            = do_nothing()
actions['gameStateMultipleActiveUpdate'] = do_nothing('  gameStateMultipleActiveUpdate')

In [ ]:
def roleSelected(state, rol_id, rol_type, unavailable_roles, **args):
    new_state = alter_dict(state)
    r = new_state['table']['roles'][rol_id]
    new_state['table']['money'] += r['money']
    r['money'] = 0
    r['selected'] = True
    return new_state

actions['roleSelected'] = roleSelected

In [ ]:
def plantationSettlement(state, player_id, **args):
    new_state = alter_dict(state)
    pl = get_player(new_state, player_id)
#     update_counter(new_state['table'], **args)
    pick_field(new_state['table'], pl, **args)
    return new_state

def pick_field(table, player, res_type, pla_id, **args):
    player['plantations'][res_type]['jobs'] += 1
    player['slots']['plantations'] -= 1
    if res_type in table['plantations']:
        table['plantations'].remove(res_type)
    plantation_lookup[pla_id] = res_type


def update_counter(target, counter_name=None, counter_delta=0, **args):
    if counter_name is not None:
        target[counter_name] += counter_delta

actions['plantationSettlement'] = plantationSettlement

In [ ]:
def discardPlantations(state, **args):
    return state

actions['discardPlantations'] = discardPlantations

In [ ]:
def drawPlantations(state, plantations_to_set, **args):
    new_state = alter_dict(state)
    new_state['table']['plantations'] = setup_plantations(plantations_to_set)
    return new_state

actions['drawPlantations'] = drawPlantations

In [ ]:
def buildingBought(state, bld_type_tr, cost, **kvargs):
    new_state, p, t = prepare(state, **kvargs)
    cost = int(cost)
        
    add_building_to_player(p, bld_type_tr, **kvargs)
    t['buildings'][bld_type_tr] -= 1
    
    p['money'] -= cost
    t['money'] += cost
    return new_state

def add_building_to_player(player, bld_type_tr, score_delta, bld_id, **kvargs):
    b = building_lookup[bld_id]
    player['slots']['buildings'] -= b['tile_size']
    player['building_vp'] += int(score_delta)
    player['potential_vp'][b['type']] += b['potential_vp']
    if b['type'] == 'commercial':
        player['buildings']['commercial'][b['name']] = False  
    else:
        player['buildings']['factories'][b['res_type']]['jobs'] += b['jobs']
    
actions['buildingBought'] = buildingBought

In [ ]:
def colonistsEarnedFromShip(state, delta, **kvargs):
    new_state, p, t = prepare(state, **kvargs)

    p['workers']['total'] += delta
    p['workers']['unemployed'] += delta
    t['workers']['arriving'] -= delta
  
    return new_state

def colonistsEarnedFromSupply(state, delta, **kvargs):
    new_state, p, t = prepare(state, **kvargs)
    
    p['workers']['total'] += delta
    p['workers']['unemployed'] += delta
    t['workers']['available'] -= delta
    
    return new_state

actions['colonistsEarnedFromSupply'] = colonistsEarnedFromSupply
actions['colonistsEarnedFromShip'] = colonistsEarnedFromShip

In [ ]:
def colonistToBuilding(state, col_id, bld_id, delta, **kvargs):
    new_state = alter_dict(state)
    colonist_lookup[col_id] = {'id': bld_id, 'type': 'bld'}
    b = building_lookup[bld_id]
    pl = get_player(new_state, **kvargs)
    pb = pl['buildings'][b['type']]

    if b['type'] != 'commercial':
        pb[b['res_type']]['workers'] += delta
    else:
        pb[b['name']] = True
    pl['workers']['unemployed'] -= delta
    return new_state

def colonistToPlantation(state, col_id, pla_id, delta, **kvargs):
    new_state = alter_dict(state)
    colonist_lookup[col_id] = {'id': pla_id, 'type': 'pla'}
    p = get_player(new_state, **kvargs)
    res_type = plantation_lookup[pla_id]
    
    p['plantations'][res_type]['workers'] += delta
    p['workers']['unemployed'] -= delta
    return new_state

def colonistToSanJuan(state, col_id, delta, **kvargs):
    id = colonist_lookup[col_id]['id']
    type = colonist_lookup[col_id]['type']
    if type == 'pla':
        state = colonistToPlantation(state, col_id, id, -delta, **kvargs)
    else:
        state = colonistToBuilding(state, col_id, id, -delta, **kvargs)
    return state
    
actions['colonistToBuilding'] = colonistToBuilding
actions['colonistToPlantation'] = colonistToPlantation
actions['colonistToSanJuan'] = colonistToSanJuan

In [ ]:
def colonistsToShip(state, delta, **kvargs):
    new_state = alter_dict(state)
    delta = int(delta)
    new_state['table']['workers']['available'] -= delta
    new_state['table']['workers']['arriving']  += delta
    return new_state

actions['colonistsToShip'] = colonistsToShip

In [ ]:
def produceGoods(state, res_type, delta, **kvargs):
    new_state, p, t = prepare(state, **kvargs)    
    delta = int(delta)    
    t['goods'][res_type] -= delta
    p['goods'][res_type] += delta
    return new_state

actions['produceGoods'] = produceGoods

In [ ]:
def doubloonsEarned(state, delta, **kvargs):
    new_state, p, t = prepare(state, **kvargs) 
    delta = int(delta)
    t['money'] -= delta
    p['money'] += delta
    return new_state

actions['doubloonsEarned'] = doubloonsEarned

In [ ]:
def doubloonToRole(state, **kvargs):
    new_state, p, t = prepare(state, **kvargs)
    rol_id = dict(kvargs['role_tooltips']).popitem()[0]
    t['money'] -= 1
    t['roles'][rol_id]['money'] += 1
    return new_state

actions['doubloonToRole'] = doubloonToRole

In [ ]:
def newGovernor(state, new_governor_player_id, **kvargs):
    new_state = alter_dict(state)
    new_state['governor'] = new_governor_player_id
    for r in new_state['table']['roles'].values():
        r['selected'] = False
    return new_state

actions['newGovernor'] = newGovernor

In [ ]:
def goodsShipped(state, shp_id, res_type, delta, **kvargs):
    new_state, p, t = prepare(state, **kvargs)
    
    if shp_id:
        t['ships'][shp_id]['res_type'] = res_type
        t['ships'][shp_id]['goods'] += delta
        p['goods'][res_type] -= delta
    else:
        t['goods'][res_type] += delta
        p['goods'][res_type] -= delta

    return new_state

def victoryPointsEarned(state, delta, **kvargs):
    new_state, p, t = prepare(state, **kvargs)
    
    t['VPs'] -= delta
    p['delivery_vp'] += delta

    return new_state

  
actions['selectShip'] = do_nothing('selectShip')
actions['selectWharf'] = do_nothing('selectWharf')
actions['goodsShipped'] = goodsShipped
actions['victoryPointsEarned'] = victoryPointsEarned

In [ ]:
def moveGoodsToSupply(state, **kvargs):
    new_state = alter_dict(state)
    
    for s in new_state['table']['ships'].values():
        if int(s['capacity']) <= s['goods']:
            new_state['table']['goods'][s['res_type']] += s['goods']
            s['goods'] = 0
            s['res_type'] = None
            
    if len(new_state['table']['stock']) == 4:
        for res_type in new_state['table']['stock']:
            new_state['table']['goods'][res_type] += 1
        new_state['table']['stock'] = []
            
    return new_state

actions['moveGoodsToSupply'] = moveGoodsToSupply

In [ ]:
def check_limit(limit):
    if run_limits[limit] == 0:
        raise Exception('"{}" must be reviewed'.format(limit))
    else:
        run_limits[limit] -= 1

In [ ]:
def goodStored(state, res_type, one_by_one, **kvargs):
    new_state, p, t = prepare(state, **kvargs)
    p['goods']['stored'][res_type] = {'one_by_one': one_by_one}
    return new_state

def goodUnstored(state, res_type, one_by_one, **kvargs):
    new_state, p, t = prepare(state, **kvargs)
       
    del p['goods']['stored'][res_type]
    return new_state

def endOfStorage(state, **kvargs):
    pprint(kvargs)
    return state

actions['goodStored'] = goodStored
actions['goodUnstored'] = goodUnstored
actions['endOfStorage'] = endOfStorage

In [ ]:
def lostGoodsToSupply(state, **kvargs):
    new_state, p, t = prepare(state, **kvargs)
    for res_type in good_types:
        if res_type not in p['goods']['stored']:
            t['goods'][res_type] += p['goods'][res_type]
            p['goods'][res_type] = 0
        elif 'one_by_one' in p['goods']['stored'][res_type] and p['goods']['stored'][res_type]['one_by_one']:
            t['goods'][res_type] += p['goods'][res_type] - 1
            p['goods'][res_type] = 1
        
    p['goods']['stored'] = {}
            
    return new_state

actions['lostGoodsToSupply'] = lostGoodsToSupply

In [ ]:
def resourceSold(state, res_type, **kvargs):
    new_state, p, t = prepare(state, **kvargs)
    
    p['goods'][res_type] -= 1
    t['stock'].append(res_type)
            
    return new_state

actions['resourceSold'] = resourceSold

In [ ]:
actions['draftCompleted'] = do_nothing('draftCompleted')
actions['reshufflePlantations'] = do_nothing('reshufflePlantations')
actions['lastYear'] = do_nothing()
actions['scoreShippingPoints'] = do_nothing()
actions['scoreConditionalPoints'] = do_nothing()
actions['scoreEndPoints'] = do_nothing()
actions['endOfGame'] = do_nothing()
actions['simpleNode'] = do_nothing()

In [ ]:
# x = debug(game_state, next_step, lostGoodsToSupply)


# ACTIVE DEVELOPMENT ZONE


In [ ]:
new_game_state = 1
while new_game_state and len(game_log) > game_state['move_id']:
        
    next_step = get_step(game_log, **game_state)
    if next_step['type'] not in ('gameStateChange', 'updateReflexionTime', 'message'):
        display(JSON(next_step, root='next_step'))
        print_line('.')
        print()
        
    new_game_state = apply_step(game_state, next_step)
    
    if new_game_state:
        if not game_state is new_game_state:
            print()    
            print_diff(game_state, new_game_state, consumer=lambda l: pprint(l, width=10))
            print_line('-')    
            
        game_state = move_to_next_step(game_log, new_game_state)
        if not game_state['sub_move_id']: 
            print()
            print_line('=')
            print('move {move_id}.{sub_move_id}'.format(**game_state))
            display(JSON(game_state, root='game_state'))
#             break          
            print_line('=')